# MaxEnt (Logistic Regression) ตั้งแต่ต้นจนจบ

1. Task Formulation
2. Feature Extraction
3. Train the model
4. Evaluation


\+ ทบทวนการเตรียมข้อมูลที่สอนไปครั้งที่แล้ว

todo
1. add previous and next word
2. add pos
3. 

In [39]:
from google.colab import  drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
import numpy as np
import glob
import sys
import os

# Load Data

In [41]:
def shift_df(df,shift,column):
    if(shift>0):
        new_column='L_'+column
    else:
        new_column='R_'+column
    df[new_column] = df[column].shift(shift)
    return df

In [42]:
def get_df(filepath):
    keyList = ['token','POS','sep','filename']
    data = {key: [] for key in keyList}
    bow= {"BOW":[],"filename":[]}
    for filename in glob.glob(os.path.join(filepath, '*.txt')):
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            #print(filename)
            column1 = []
            for i,line in enumerate(f.readlines()):
                entry = line.rstrip('\n').split('\t')
                #print(entry)
                #print(i)
                if(len(entry)!=4):
                    entry = [' ','PU','SP','SP']
                column1.append(str(entry[0]).replace('_',' '))
                #replace _ with space
                data["token"].append(str(entry[0]).replace('_',' '))
                data["POS"].append(str(entry[1]))
                if(entry[3]=='SP'):
                    data["sep"].append('SEP')
                else:               
                    data["sep"].append('N')
                data["filename"].append(filename[-10:])
            bow["BOW"].append(['|'.join(column1)])
            bow["filename"].append(filename[-10:])
    bow_df = pd.DataFrame.from_dict(bow)
    bow_df.filename.astype(str)
    df=pd.DataFrame.from_dict(data)
    df = df.merge(bow_df,on='filename')
    # df.join(bow_df,on='filename')
    # df['n_tokens'] =df.tokens.apply(len)
    df = shift_df(df,1,'POS')
    df = shift_df(df,-1,'POS')
    df.fillna('PU', inplace=True)
    df = shift_df(df,1,'token')
    df = shift_df(df,-1,'token')
    df.fillna(' ', inplace=True)
  
    return df

In [43]:
train_df=get_df('/content/drive/My Drive/LST_20/train/')
test_df=get_df('/content/drive/My Drive/LST_20/test/')
eval_df=get_df('/content/drive/My Drive/LST_20/eval/')

In [44]:
# eval_df = shift_df(eval_df,1,'POS')
# eval_df = shift_df(eval_df,-1,'POS')
# eval_df.fillna('SP', inplace=True)
# eval_df = shift_df(eval_df,1,'token')
# eval_df = shift_df(eval_df,-1,'token')
# eval_df.fillna(' ', inplace=True)
eval_df

,token,POS,sep,filename,BOW,L_POS,R_POS,L_token,R_token
0,กมธ.,NN,N,T13176.txt,"[กมธ.|สอบ|""|พอเพียง|""|เล็ง|หา|ข้อมูล|หุ้น|โยง|...",PU,VV,,สอบ
1,สอบ,VV,N,T13176.txt,"[กมธ.|สอบ|""|พอเพียง|""|เล็ง|หา|ข้อมูล|หุ้น|โยง|...",NN,PU,กมธ.,""""
2,"""",PU,N,T13176.txt,"[กมธ.|สอบ|""|พอเพียง|""|เล็ง|หา|ข้อมูล|หุ้น|โยง|...",VV,NN,สอบ,พอเพียง
3,พอเพียง,NN,N,T13176.txt,"[กมธ.|สอบ|""|พอเพียง|""|เล็ง|หา|ข้อมูล|หุ้น|โยง|...",PU,PU,"""",""""
4,"""",PU,N,T13176.txt,"[กมธ.|สอบ|""|พอเพียง|""|เล็ง|หา|ข้อมูล|หุ้น|โยง|...",NN,VV,พอเพียง,เล็ง
...,...,...,...,...,...,...,...,...,...
246475,"""",PU,N,T12123.txt,"[""|ณวัฒน์|""|ยก|ทัพ| |8| |นัก|แสดง|ใช้|ชีวิต|ที...",PA,PU,ครับ,
246476,,PU,N,T12123.txt,"[""|ณวัฒน์|""|ยก|ทัพ| |8| |นัก|แสดง|ใช้|ชีวิต|ที...",PU,NN,"""",ณวัฒน์
246477,ณวัฒน์,NN,N,T12123.txt,"[""|ณวัฒน์|""|ยก|ทัพ| |8| |นัก|แสดง|ใช้|ชีวิต|ที...",PU,VV,,กล่าว
246478,กล่าว,VV,N,T12123.txt,"[""|ณวัฒน์|""|ยก|ทัพ| |8| |นัก|แสดง|ใช้|ชีวิต|ที...",NN,PU,ณวัฒน์,


# Train-dev-test split


In [45]:
train_df.shape

(2778036, 9)

In [46]:
eval_df.shape

(246480, 9)

In [47]:
test_df.shape

(212528, 9)

# EDA

In [48]:
# train_df.to_csv('lst20sentseg-train.csv',index=False)
# eval_df.to_csv('lst20sentseg-eval.csv',index=False)
# test_df.to_csv('lst20sentseg-test.csv',index=False)

In [49]:
all_df = pd.concat([train_df,test_df,eval_df])

In [50]:
all_df.head()

,token,POS,sep,filename,BOW,L_POS,R_POS,L_token,R_token
0,อ๊อฟ,NN,N,T11879.txt,[อ๊อฟ|โต้|ข่าว|เลิก|กับ|แฟน|เพื่อ|มา|คั่ว|จั๊ก...,PU,VV,,โต้
1,โต้,VV,N,T11879.txt,[อ๊อฟ|โต้|ข่าว|เลิก|กับ|แฟน|เพื่อ|มา|คั่ว|จั๊ก...,NN,NN,อ๊อฟ,ข่าว
2,ข่าว,NN,N,T11879.txt,[อ๊อฟ|โต้|ข่าว|เลิก|กับ|แฟน|เพื่อ|มา|คั่ว|จั๊ก...,VV,VV,โต้,เลิก
3,เลิก,VV,N,T11879.txt,[อ๊อฟ|โต้|ข่าว|เลิก|กับ|แฟน|เพื่อ|มา|คั่ว|จั๊ก...,NN,PS,ข่าว,กับ
4,กับ,PS,N,T11879.txt,[อ๊อฟ|โต้|ข่าว|เลิก|กับ|แฟน|เพื่อ|มา|คั่ว|จั๊ก...,VV,NN,เลิก,แฟน


In [51]:
sep_df = all_df[all_df.sep=='SEP']

In [52]:
sep_df.shape

(74180, 9)

In [53]:
L_token = sep_df.groupby(['L_token','L_POS']).count()['token'].sort_values(ascending=False)
L_token.to_csv('/content/drive/My Drive/L_token_report.csv')
L_token

L_token      L_POS
"            PU       2147
กล่าว        VV       1710
นี้          AJ       1674
ได้          AX       1488
แล้ว         AX       1468
                      ... 
พงส.         NN          1
พงษ์ไพบูลย์  NN          1
พงษ์สาร      NN          1
พงศ์สุชาติ   NN          1
…            PU          1
Name: token, Length: 8416, dtype: int64

In [54]:
R_token = sep_df.groupby(['R_token','R_POS']).count()['token'].sort_values(ascending=False)
R_token.to_csv('/content/drive/My Drive/R_token_report.csv')
R_token

R_token            R_POS
"                  PU       5310
นาย                NN       4471
โดย                CC       3460
ส่วน               CC       2649
และ                CC       2098
                            ... 
วินิจฉัย           VV          1
วินโดว์ส7          NN          1
วิมเบิล            NN          1
วิรัช              NN          1
สำนักงานเลขานุการ  NN          1
Name: token, Length: 4790, dtype: int64

# สร้าง Feature bag-of-words


In [55]:
def featurize(token,L_token,R_token,POS,L_POS,R_POS):
    #token_list = token_list[0].split('|')
    features = {}
    # for token in token_list:
    #     try:
    #         features[token]+=1 
    #     except:
    #         features[token] = 1
    features['current_token'] = token
    features['left_token'] = L_token
    features['right_token'] = R_token
    features['current_POS'] = POS
    features['left_POS'] = L_POS
    features['right_POS'] = R_POS
    return features

In [56]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
vectorizer = DictVectorizer(sparse=True)
# train_features = train_df[['token','L_token','R_token']].apply(featurize)
train_features = train_df.apply(lambda x: featurize(x.token, x.L_token,x.R_token,x.POS, x.L_POS,x.R_POS), axis=1)
feature_vectors = vectorizer.fit_transform(train_features)

In [57]:
train_features[0:10]

0    {'current_token': 'อ๊อฟ', 'left_token': ' ', '...
1    {'current_token': 'โต้', 'left_token': 'อ๊อฟ',...
2    {'current_token': 'ข่าว', 'left_token': 'โต้',...
3    {'current_token': 'เลิก', 'left_token': 'ข่าว'...
4    {'current_token': 'กับ', 'left_token': 'เลิก',...
5    {'current_token': 'แฟน', 'left_token': 'กับ', ...
6    {'current_token': 'เพื่อ', 'left_token': 'แฟน'...
7    {'current_token': 'มา', 'left_token': 'เพื่อ',...
8    {'current_token': 'คั่ว', 'left_token': 'มา', ...
9    {'current_token': 'จั๊กจั่น', 'left_token': 'ค...
dtype: object

In [58]:
feature_vectors

<2778036x146874 sparse matrix of type '<class 'numpy.float64'>'
	with 16668216 stored elements in Compressed Sparse Row format>

In [59]:
# vectorizer.get_feature_names()

In [60]:
#  vectorizer.vocabulary_

In [61]:
# tokens = train['title tokens'][100]
# print(tokens)
# [vectorizer.vocabulary_[x] for x in tokens]

In [62]:
feature_vectors[0]

<1x146874 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [63]:
train_label=train_df['sep']

In [64]:
lr = LogisticRegression(dual = False, solver='lbfgs')
lr.fit(feature_vectors, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [65]:
lr.coef_.shape

(1, 146874)

In [66]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
dev_featurized_list_dicts = eval_df.apply(lambda x: featurize(x.token, x.L_token,x.R_token,x.POS, x.L_POS,x.R_POS), axis=1)
dev_feature_vector = vectorizer.transform(dev_featurized_list_dicts)
dev_feature_vector

<246480x146874 sparse matrix of type '<class 'numpy.float64'>'
	with 1465074 stored elements in Compressed Sparse Row format>

In [67]:
eval_df['sep'][0:50]

0       N
1       N
2       N
3       N
4       N
5       N
6       N
7       N
8       N
9       N
10      N
11      N
12      N
13      N
14    SEP
15      N
16      N
17      N
18      N
19      N
20      N
21      N
22      N
23      N
24      N
25      N
26      N
27      N
28      N
29      N
30      N
31      N
32      N
33      N
34      N
35      N
36      N
37      N
38      N
39      N
40      N
41      N
42      N
43      N
44      N
45      N
46      N
47      N
48      N
49      N
Name: sep, dtype: object

In [68]:
predictions = lr.predict(dev_feature_vector)
predictions[0:50]

array(['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'SEP', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N'],
      dtype=object)

In [69]:
print(accuracy_score(eval_df['sep'], predictions))

0.982838364167478


In [70]:
report = classification_report(eval_df['sep'], predictions,output_dict=True)


In [71]:
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
N,0.986165,0.996417,0.991264,240860.000000
SEP,0.723042,0.400890,0.515797,5620.000000
accuracy,0.982838,0.982838,0.982838,0.982838
macro avg,0.854604,0.698653,0.753531,246480.000000
weighted avg,0.980165,0.982838,0.980423,246480.000000


In [72]:
print (confusion_matrix(eval_df['sep'], predictions))

[[239997    863]
 [  3367   2253]]


In [73]:
lr.coef_.shape

(1, 146874)

In [74]:
report_df.to_csv('/content/drive/My Drive/MaxEnt sent seg SP classifier--3gramPOS.csv')